In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from config import gkey




In [52]:
fire_df = pd.read_csv ('Fire.csv')
fire_df.head()

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,TOTAL VALUE,STRUCTURE LOSS,CONTENTS LOSS,TOTAL SAVED,PERCENTAGE LOSS,SPRINKER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,...,166370.0,-5000.0,0.0,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE,32.732107,-96.814783
1,1,17962.0,10/30/2019,03:42 AM,75229.0,7,A,13.0,DWELLING,1.0,...,535470.0,-10000.0,0.0,525470.0,1.87,NaN,NaN,10728 LATHROP DR,32.894248,-96.857108
2,2,17963.0,10/30/2019,10:47 AM,75241.0,5,B,3.0,DWELLING,1.0,...,306000.0,-5000.0,-15000.0,286000.0,6.54,NaN,NaN,6205 S R L THORNTON SERV,32.694543,-96.824020
3,3,17964.0,10/30/2019,05:32 PM,75243.0,2,B,10.0,DWELLING,1.0,...,126710.0,-8000.0,-4000.0,114710.0,9.47,NaN,NaN,12044 WILLOWDELL DR,32.913584,-96.764435
4,4,17965.0,10/30/2019,11:03 AM,75212.0,9,B,6.0,DWELLING,1.0,...,84000.0,-45000.0,-10000.0,29000.0,65.48,NaN,NaN,2212 LAPSLEY ST,32.769986,-96.908816


In [65]:
fire_df['TOTAL LOSS'] = fire_df['TOTAL VALUE'] - fire_df['TOTAL SAVED']
fire_df = fire_df.rename(columns={
    'SPRINKER SYSTEM':'SPRINKLER SYSTEM'
})



In [66]:
fire_df['ALARM SYSTEM'] = fire_df['ALARM SYSTEM'].replace({
    'UNK': None,
    'UKN': None
})

fire_df['ALARM SYSTEM'].unique()

array([nan, 'YES', 'NO', None], dtype=object)

In [67]:
fire_df['SPRINKLER SYSTEM'] = fire_df['SPRINKLER SYSTEM'].replace({
    'UNK': None
})

fire_df['SPRINKLER SYSTEM'].unique()

array([nan, 'YES', 'NO', None], dtype=object)

In [77]:
fire_df.to_csv('Fire.csv', index = False)

In [108]:
alarm_columns = fire_df[['TOTAL SAVED', 'PERCENTAGE LOSS', 'SPRINKLER SYSTEM', 'ALARM SYSTEM']]
alarm_columns = alarm_columns.dropna()
alarm_group = alarm_columns.groupby(['SPRINKLER SYSTEM','ALARM SYSTEM'])
alarm_describe = alarm_group.describe()
alarm_mean = alarm_group.mean()
alarm_describe

TOTAL SAVED                              \
                                    count          mean           std   
SPRINKLER SYSTEM ALARM SYSTEM                                           
NO               NO                 161.0  1.732122e+06  4.546680e+06   
                 YES                181.0  8.120217e+06  5.366334e+07   
YES              NO                   3.0  1.305067e+06  1.580012e+06   
                 YES                 66.0  2.897127e+07  9.108470e+07   

                                                                          \
                                   min        25%        50%         75%   
SPRINKLER SYSTEM ALARM SYSTEM                                              
NO               NO              450.0    47240.0   118000.0    658060.0   
                 YES            3500.0   120800.0   409150.0   5367020.0   
YES              NO            65900.0   415450.0   765000.0   1924650.0   
                 YES           24500.0  2162050.0  8901100.0  27330660.0   

                                           PERCENTAGE LOSS             \
                                       max           count       mean   
SPRINKLER SYSTEM ALARM SYSTEM                                           
NO               NO             29997530.0           161.0  16.567019   
                 YES           719999980.0           181.0   8.645470   
YES              NO              3084300.0             3.0  14.466667   
                 YES           726499700.0            66.0   1.632727   

                                                                            
                                     std   min    25%   50%     75%    max  
SPRINKLER SYSTEM ALARM SYSTEM                                               
NO               NO            22.648835  0.00  0.400  4.17  25.000  86.74  
                 YES           15.651785  0.00  0.070  0.97   9.280  94.58  
YES              NO            23.707826  0.08  0.785  1.49  21.660  41.83  
                 YES            7.040789  0.00  0.000  0.02   0.205  48.02

In [109]:
alarm_columns['SYSTEM INSTALLED'] = ' '


In [111]:
alarm_columns['SYSTEM INSTALLED'].loc[
    (alarm_columns['SPRINKLER SYSTEM'] == 'NO') &
    (alarm_columns['ALARM SYSTEM'] == 'NO')
] = 'Neither'

alarm_columns['SYSTEM INSTALLED'].loc[
    (alarm_columns['SPRINKLER SYSTEM'] == 'NO') &
    (alarm_columns['ALARM SYSTEM'] == 'YES')
] = 'Alarm Only'

alarm_columns['SYSTEM INSTALLED'].loc[
    (alarm_columns['SPRINKLER SYSTEM'] == 'YES') &
    (alarm_columns['ALARM SYSTEM'] == 'NO')
] = 'Sprinkler Only'

alarm_columns['SYSTEM INSTALLED'].loc[
    (alarm_columns['SPRINKLER SYSTEM'] == 'YES') &
    (alarm_columns['ALARM SYSTEM'] == 'YES')
] = 'Both'

alarm_columns

C:\Users\Rob\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,TOTAL SAVED,PERCENTAGE LOSS,SPRINKLER SYSTEM,ALARM SYSTEM,SYSTEM INSTALLED
11,2148550.0,0.92,YES,YES,Both
13,8898900.0,0.01,YES,YES,Both
15,4831220.0,0.02,NO,YES,Alarm Only
16,98790.0,1.00,NO,NO,Neither
17,32450.0,58.10,NO,NO,Neither
...,...,...,...,...,...
613,3217610.0,0.04,NO,YES,Alarm Only
615,376090.0,0.00,YES,YES,Both
616,243500.0,6.70,NO,YES,Alarm Only
619,900.0,40.00,NO,NO,Neither
